Insper

# Aula 13 (ou 14 ou 17 =P) - Introdução à GPU

Pessoal, podemos usar a GPU do Colab para os códigos iniciais de familiarização com a biblioteca [Thrust](https://thrust.github.io/).

**Passos**:
1. Criar o ambiente de execução
2. Alterar o ambiente de execução para T4 (GPU)
3. Verificar a versão instalada do `nvcc` (comando abaixo)
4. Programar numa célula e salvar o conteúdo como arquivo ".cu"
5. Compilar com `nvcc` (comando abaixo)
6. Executar normalmente!

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


```cpp
#include <thrust/device_vector.h>
#include <thrust/host_vector.h>
#include <iostream>

int main() {
    thrust::host_vector<double> host(5, 0);
    host[4] = 35;

    /* na linha abaixo os dados são copiados
       para GPU */
    thrust::device_vector<double> dev(host);
    /* a linha abaixo só muda o vetor na CPU */
    host[2] = 12;

    printf("Host vector: ");
    for (auto i = host.begin(); i != host.end(); i++) {
        std::cout << *i << " "; // este acesso é rápido -- CPU
    }
    printf("\n");

    printf("Device vector: ");
    for (auto i = dev.begin(); i != dev.end(); i++) {
        std::cout << *i << " "; // este acesso é lento! -- GPU
    }
    printf("\n");

}
```

In [ ]:
%%writefile exemplo1.cu
#include <thrust/device_vector.h>
#include <thrust/host_vector.h>
#include <iostream>

int main() {
    thrust::host_vector<double> host(5, 0);
    host[4] = 35;

    /* na linha abaixo os dados são copiados para GPU */
    thrust::device_vector<double> dev(host);
    
    /* a linha abaixo só muda o vetor na CPU */
    host[2] = 12;

    printf("Host vector: ");
    for (auto i = host.begin(); i != host.end(); i++) {
        std::cout << *i << " "; // este acesso é rápido -- CPU
    }
    printf("\n");

    printf("Device vector: ");
    for (auto i = dev.begin(); i != dev.end(); i++) {
        std::cout << *i << " "; // este acesso é lento! -- GPU
    }
    printf("\n");

}

Writing exemplo1.cu


In [ ]:
# Mesmo comando do roteiro
!nvcc -arch=sm_70 -std=c++14 exemplo1.cu -o exemplo1

In [ ]:
!./exemplo1

Host vector: 0 0 12 0 35 
Device vector: 0 0 0 0 35 


```cpp
%%writefile exemplo2.cu
#include <thrust/device_vector.h>
#include <thrust/host_vector.h>
#include <thrust/functional.h>
#include <thrust/transform.h>
#include <iostream>

int main() {
    thrust::device_vector<double> V1(10, 0);
    thrust::sequence(V1.begin(), V1.end());

    thrust::device_vector<double> V2(10, 0);
    thrust::fill(V2.begin(), V2.begin() + 5, 5.5);
    thrust::fill(V2.begin()+5, V2.end(), 10);

    thrust::device_vector<double> V3(10);
    thrust::device_vector<double> V4(10);

    thrust::transform(V1.begin(), V1.end(), V2.begin(), V3.begin(), thrust::plus<double>());
    thrust::transform(V1.begin(), V1.end(), thrust::constant_iterator<double>(0.5), V4.begin(), thrust::multiplies<double>());

    printf("V1: ");
    for (thrust::device_vector<double>::iterator i = V1.begin(); i != V1.end(); i++) {
        std::cout << *i << " ";
    }
    printf("\n");

    printf("V2: ");
    for (thrust::device_vector<double>::iterator i = V2.begin(); i != V2.end(); i++) {
        std::cout << *i << " ";
    }
    printf("\n");

    printf("V3: ");
    for (thrust::device_vector<double>::iterator i = V3.begin(); i != V3.end(); i++) {
        std::cout << *i << " ";
    }
    printf("\n");

    printf("V4: ");
    for (thrust::device_vector<double>::iterator i = V4.begin(); i != V4.end(); i++) {
        std::cout << *i << " ";
    }
    printf("\n");

}

```

In [ ]:
%%writefile exemplo2.cu
#include <thrust/device_vector.h>
#include <thrust/host_vector.h>
#include <thrust/functional.h>
#include <thrust/transform.h>
#include <iostream>

int main() {
    thrust::device_vector<double> V1(10, 0);
    thrust::sequence(V1.begin(), V1.end()); // 0, 1, 2, 3, 4, 5, 6, 7, 8, 9

    thrust::device_vector<double> V2(5, 0);
    thrust::fill(V2.begin(), V2.begin() + 2, 5.5); // 5.5, 5.5, 0, 0, 0
    thrust::fill(V2.begin()+2, V2.end(), 10); // 5.5, 5.5, 10, 10, 10

    thrust::device_vector<double> V3(10);
    //thrust::device_vector<double> V4(10);

    thrust::transform(V1.begin(), V1.end(), V2.begin(), V3.begin(), thrust::plus<double>());
    //thrust::transform(V1.begin(), V1.end(), thrust::constant_iterator<double>(0.5), V4.begin(), thrust::multiplies<double>());

    printf("V1: ");
    for (thrust::device_vector<double>::iterator i = V1.begin(); i != V1.end(); i++) {
        std::cout << *i << " ";
    }
    printf("\n");

    printf("V2: ");
    for (thrust::device_vector<double>::iterator i = V2.begin(); i != V2.end(); i++) {
        std::cout << *i << " ";
    }
    printf("\n");

    printf("V3: ");
    for (thrust::device_vector<double>::iterator i = V3.begin(); i != V3.end(); i++) {
        std::cout << *i << " ";
    }
    printf("\n");

    //printf("V4: ");
    //for (thrust::device_vector<double>::iterator i = V4.begin(); i != V4.end(); i++) {
    //    std::cout << *i << " ";
    //}
    printf("\n");

}


Overwriting exemplo2.cu


In [ ]:
!nvcc -arch=sm_70 -std=c++14 exemplo2.cu -o exemplo2

In [ ]:
!./exemplo2

V1: 0 1 2 3 4 5 6 7 8 9 
V2: 5.5 5.5 10 10 10 
V3: 5.5 6.5 12 13 14 5 6 7 8 9 



```cpp
%%writefile stocks.cu
#include<thrust/device_vector.h>
#include <thrust/host_vector.h>
#include <iostream>
#include <chrono>
using namespace std;
int main() {
    int n =  2518;
    double value = 0.0;
    std::chrono::duration<double> diff;

    auto leitura_i = std::chrono::steady_clock::now();
    thrust::host_vector<double> host(n,0);
    for(int i = 0; i<n; i++){
        cin >> value;
        host[i] = value;
    }
    auto leitura_f = std::chrono::steady_clock::now();
    diff  = leitura_f-leitura_i;
    cout << "Tempo de LEITURA (em segundos)  " << diff.count() << endl;


    auto copia_i = std::chrono::steady_clock::now();
    thrust::device_vector<double> dev(host);
    auto copia_f = std::chrono::steady_clock::now();
    diff  = copia_f-copia_i;
    cout << "Tempo de CÓPIA (em segundos)  " << diff.count() << endl;
}

```

In [ ]:
%%writefile stocks.cu
#include<thrust/device_vector.h>
#include <thrust/host_vector.h>
#include <iostream>
#include <chrono>
using namespace std;
int main() {
    int n =  2518;
    double value = 0.0;
    std::chrono::duration<double> diff;

    auto leitura_i = std::chrono::steady_clock::now();
    thrust::host_vector<double> host(n,0);
    for(int i = 0; i<n; i++){
        cin >> value;
        host[i] = value;
    }
    auto leitura_f = std::chrono::steady_clock::now();
    diff  = leitura_f-leitura_i;
    cout << "Tempo de LEITURA (em segundos)  " << diff.count() << endl;


    auto copia_i = std::chrono::steady_clock::now();
    thrust::device_vector<double> dev(host);
    auto copia_f = std::chrono::steady_clock::now();
    diff  = copia_f-copia_i;
    cout << "Tempo de CÓPIA (em segundos)  " << diff.count() << endl;
}

Writing stocks.cu


In [ ]:
!nvcc -arch=sm_70 -std=c++14 stocks.cu -o stocks

In [ ]:
!./stocks < stocks.txt

Tempo de LEITURA (em segundos)  0.00294546
Tempo de CÓPIA (em segundos)  0.198211
